# 🧠 使用ChatGPT嵌入和NMF进行主题建模

本笔记本演示了一个现代无监督主题建模流程：

- ✅ 使用OpenAI的`text-embedding-3-small`模型生成文本嵌入
- ✅ 使用NMF(非负矩阵分解)进行主题聚类
- ✅ 两种主题解释方法：
    - TF-IDF关键词提取
    - 基于ChatGPT的摘要生成

这种方法提供了语义主题聚类和人类可读的摘要。

In [ ]:
import openai
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

# 设置您的OpenAI API密钥
# openai.api_key = "OpenAI_Key"  # 替换为您的真实API密钥

# 示例数据集
data = pd.DataFrame({
    'text': [
        "经济正面临通胀和加息带来的挑战",
        "由于对经济衰退的担忧，股市一直波动不定",
        "投资者正在寻找黄金和债券等避险资产",
        "机器学习算法正在快速改进",
        "神经网络被广泛应用于图像识别任务",
        "人工智能正在改变医疗保健行业",
        "新款iPhone发布，电池和摄像头更好",
        "三星最新手机采用可折叠屏幕",
        "科技小玩意是节日期间的热门礼物",
        "科学家在宜居带发现了一颗新的系外行星"
    ]
})

## 第一步：生成ChatGPT嵌入向量

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

# 获取嵌入向量
embedding_list = []
for i, row in data.iterrows():
    embedding = get_embedding(row['text'])
    embedding_list.append(embedding)

X = np.array(embedding_list)  # 形状 (文档数, 1536)

## 第二步：降维(PCA) + 主题建模(NMF)

In [ ]:
# 降维处理
X_reduced = PCA(n_components=100, random_state=42).fit_transform(X)

# 主题建模
n_topics = 3
nmf = NMF(n_components=n_topics, random_state=42)
W = nmf.fit_transform(X_reduced)
H = nmf.components_

# 为每个文档分配主要主题
topic_assignments = np.argmax(W, axis=1)
data['topic'] = topic_assignments

# 显示结果
data[['text', 'topic']]

## 第三步a：每个主题的关键词提取(TF-IDF + NMF)

In [ ]:
print("=== TF-IDF + NMF关键词摘要 ===")
n_keywords = 5

for topic_id in sorted(data['topic'].unique()):
    topic_texts = data[data['topic'] == topic_id]['text'].tolist()
    print(f"\n--- 主题 {topic_id} ---")

    # TF-IDF
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, stop_words='english')
    tfidf = vectorizer.fit_transform(topic_texts)

    # NMF
    nmf_sub = NMF(n_components=1, random_state=0)
    W_sub = nmf_sub.fit_transform(tfidf)
    H_sub = nmf_sub.components_

    feature_names = vectorizer.get_feature_names_out()
    top_keywords = [feature_names[i] for i in H_sub[0].argsort()[::-1][:n_keywords]]
    print("Top keywords:", ", ".join(top_keywords))

## 第三步b：通过ChatGPT生成主题摘要

In [ ]:
print("=== 基于ChatGPT的主题摘要 ===")
for topic_id in sorted(data['topic'].unique()):
    examples = data[data['topic'] == topic_id]['text'].tolist()
    prompt = "总结以下句子的主题:\n\n" + "\n\n".join(examples)

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    summary = response.choices[0].message.content
    print(f"\n--- 主题 {topic_id} 摘要 ---\n{summary}")

## ✅ 总结

本笔记本展示了如何：

- 使用ChatGPT为短文本创建有意义的嵌入
- 使用NMF进行语义聚类
- 使用以下方法解释主题：
    - 传统的TF-IDF关键词提取
    - 基于LLM的摘要生成

这种方法结合了LLM的强大能力和可解释性，不依赖单纯的词频统计就能实现最先进的主题建模。